# Chapter11.SparkML

### 示例：垃圾邮件分类

In [1]:
from pyspark import SparkContext
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.classification import LogisticRegressionWithSGD

sc = SparkContext("local[*]","SparkML")
spam = sc.textFile("file:///Users/lixiwei-mac/Documents/git/learning-spark/files/rubbish.txt")
normal = sc.textFile("file:///Users/lixiwei-mac/Documents/git/learning-spark/files/normal.txt")

In [2]:
# 创建一个HashingTF实例来把文本映射为包含10000个特征的向量
tf = HashingTF(numFeatures=10000)
# 各邮件都被切分为单词，每个单词都被映射为一个特征
spamFeatures = spam.map(lambda email : tf.transform(email.split(" ")))
print(spamFeatures.collect())
normalFeatures = normal.map(lambda email : tf.transform(email.split(" ")))
print(normalFeatures.collect())

[SparseVector(10000, {830: 1.0, 3289: 1.0, 3870: 1.0, 5542: 1.0, 6299: 1.0}), SparseVector(10000, {5542: 4.0}), SparseVector(10000, {2744: 1.0, 5542: 1.0, 5706: 1.0}), SparseVector(10000, {0: 1.0, 1363: 1.0, 5542: 1.0, 6554: 1.0, 9466: 1.0, 9740: 1.0}), SparseVector(10000, {0: 1.0}), SparseVector(10000, {2403: 1.0, 3233: 1.0, 3870: 1.0, 5542: 1.0})]
[SparseVector(10000, {830: 1.0, 3289: 1.0, 3870: 1.0, 4842: 1.0, 6931: 1.0}), SparseVector(10000, {2678: 1.0, 2744: 1.0, 6931: 1.0, 9241: 1.0}), SparseVector(10000, {1499: 1.0, 1518: 1.0, 1583: 1.0, 2403: 1.0, 2744: 1.0, 3317: 1.0, 3870: 1.0, 4757: 1.0, 6554: 1.0, 6931: 1.0}), SparseVector(10000, {1136: 1.0, 2438: 1.0, 6052: 1.0, 6931: 1.0, 8977: 1.0}), SparseVector(10000, {6931: 4.0})]


In [3]:
# 创建LabeledPoint数据集分别存放 垃圾邮件 和 正常邮件 的例子
postiveExamples = spamFeatures.map(lambda features : LabeledPoint(0,features))
negativeExamples = normalFeatures.map(lambda features : LabeledPoint(1,features))
trainingData = postiveExamples.union(negativeExamples)
trainingData.cache()# 因为逻辑回归是迭代算法，所以缓存训练数据RDD

UnionRDD[8] at union at NativeMethodAccessorImpl.java:-2

In [4]:
# 使用SGD算法运行逻辑回归
model = LogisticRegressionWithSGD.train(trainingData)

In [11]:
# 以 垃圾邮件 和 正常邮件 的例子分别进行测试
# 首先使用一样的HashingTF特征来得到特征向量，然后对该向量应用得到的模型
posTest = tf.transform("rubbish rubbish rubbish rubbish rubbish rubbish rubbish rubbish ".split())
negTest = tf.transform("normal normal normal normal normal normal normal".split())
print("Prediction for postive test example: %g" % model.predict(posTest))
print("Prediction for negative test example: %g" % model.predict(negTest))

Prediction for postive test example: 0
Prediction for negative test example: 0


### 4.数据类型
- 向量

In [37]:
from numpy import array
from pyspark.mllib.linalg import Vectors
# 创建稠密向量 <1.0,2.0,3.0>
denseVec1 = array([1.0,2.0,3.0,0.0]) # Numpy 数组可以直接传给Mllib
denseVec2 = Vectors.dense([1.0,2.0,3.0,0.0]) # 或者使用Vectors类来创建
print(denseVec1)
print(denseVec2)
# 创建稀疏向量 该方法直接收向量的维度以及非0位的位置和对应的值
# 这些数据可以用一个dict来传递，或者使用两个分别代表位置和值的list
sparseVec1 = Vectors.sparse(4,{0:1.0,1:2.0,3:0})
sparseVec2 = Vectors.sparse(4,[0,2,5],[1.0,2.0,0.0])
print(sparseVec1)
print(sparseVec2)

[ 1.  2.  3.  0.]
[1.0,2.0,3.0,0.0]
(4,[0,1,3],[1.0,2.0,0.0])
(4,[0,2,5],[1.0,2.0,0.0])


### 5.算法

In [18]:
# TF-IDF
from pyspark.mllib.feature import HashingTF,IDF
sentence = "hello hello world"
rdd = sc.wholeTextFiles("file:///Users/lixiwei-mac/Documents/git/learning-spark/files/rubbish.txt")
words = sentence.split()
tf = HashingTF(10000)
tfVectors = tf.transform(rdd).cache()

# 计算IDF,然后计算TF-IDF向量
idf = IDF()
idfModel = idf.fit(tfVectors)
tfIdfVectors = idfModel.transform(tfVectors)
print(tfIdfVectors.collect())

[SparseVector(10000, {5947: 0.0, 9861: 0.0})]


#### 缩放
控制平均值 和 标准差

In [30]:
from pyspark.mllib.feature import StandardScaler,Vectors,Normalizer,Word2Vec
vectors = [Vectors.dense([1,2,3]),Vectors.dense([5,6,7])]
dataset = sc.parallelize(vectors)
scaler = StandardScaler(withMean=True,withStd=True)
model = scaler.fit(dataset)
result = model.transform(dataset)
result.collect()

[DenseVector([-0.7071, -0.7071, -0.7071]),
 DenseVector([0.7071, 0.7071, 0.7071])]

#### 正规化

In [29]:
Normalizer.transform(dataset)

TypeError: transform() missing 1 required positional argument: 'vector'

#### Word2Vec

In [35]:
model = Word2Vec.fit(dataset)
result = model.transform(dataset)
result.collect()

TypeError: fit() missing 1 required positional argument: 'data'

### 分类与回归
- 线性回归
- 逻辑回归
- 支持向量机
- 朴素贝叶斯
- 决策树与随机森林

In [ ]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.regression import LinearRegressionWithSGD
points = 